In [1]:
import numpy as np

In [4]:
rows = np.loadtxt('2020-04-01_lotto_v904_data.csv',delimiter=",")
print(len(rows))

904


In [8]:
def number2onehot(number):
    one_hot = np.zeros(45)

    for i in range(6):
        one_hot[int(number[i])-1] = 1
        
    return one_hot

In [9]:
def onehot2number(onehot):
    number = []
    
    for i in range(len(onehot)):
        if onehot[i] == 1:
            number.append(i+1)
        
    return number

In [10]:
numbers = rows[:,1:7]

In [15]:
onehots = list(map(number2onehot,numbers))

In [17]:
x_samples = onehots[0:904-1]
y_samples = onehots[1:904]

In [20]:
#원핫인코딩으로 표시
print("ohbins")
print("X[0]: " + str(x_samples[0]))
print("Y[0]: " + str(y_samples[0]))

#번호로 표시
print("numbers")
print("X[0]: " + str(onehot2number(x_samples[0])))
print("Y[0]: " + str(onehot2number(y_samples[0])))

ohbins
X[0]: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
Y[0]: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
numbers
X[0]: [10, 23, 29, 33, 37, 40]
Y[0]: [9, 13, 21, 25, 32, 42]


# 데이터셋 구성

In [23]:
train_idx = (0,800)
val_idx = (800,850)
test_idx = (850, len(numbers))

print("train: {0}, val: {1}, test: {2}".format(train_idx, val_idx, test_idx))

train: (0, 800), val: (800, 850), test: (850, 904)


In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
train_loss = []
train_acc = []
val_loss = []
val_acc = []

for epoch in range(100):
    model.reset_states()

    batch_train_loss = []
    batch_train_acc = []

    for i in range(train_idx[0],train_idx[1]):
        xs = x_samples[i].reshape(1,1,45)
        ys = y_samples[i].reshape(1,45)

        loss, acc = model.train_on_batch(xs,ys)

        batch_train_acc.append(acc)
        batch_train_loss.append(loss)

        train_acc.append(np.mean(batch_train_acc))
        train_loss.append(np.mean(batch_train_loss))

    batch_val_acc = []
    batch_val_loss = []

    for i in range(val_idx[0], val_idx[1]):
        xs = x_samples[i].reshape(1,1,45)
        ys = y_samples[i].reshape(1,45)

        loss, acc = model.test_on_batch(xs,ys)

        batch_val_acc.append(acc)
        batch_val_loss.append(loss)

        val_acc.append(np.mean(batch_val_acc))
        val_loss.append(np.mean(batch_val_loss))

    print('epoch {0:4d} train acc {1:0.3f} loss {2:0.3f} val acc {3:0.3f} loss {4:0.3f}'.format(epoch, np.mean(batch_train_acc), np.mean(batch_train_loss), np.mean(batch_val_acc), np.mean(batch_val_loss)))

    model.save('model_{0:04d}.h5'.format(epoch+1))

In [ ]:
train_loss = []
train_acc = []

for epoch in range(200):
    model.reset_states()

    batch_train_acc = []
    batch_train_loss = []

    for i in range(len(x_samples)):
        xs = x_samples[i].reshape(1,1,45)
        ys = y_samples[i].reshape(1,45)

        loss, acc = model.train_on_batch(xs, ys)

        batch_train_acc.append(acc)
        batch_train_loss.append(loss)

    train_acc.append(np.mean(batch_train_acc))
    train_loss.append(np.mean(batch_train_loss))

    if epoch % 10 == 0:
    print('epoch {0:4d} train acc {1:0.3f} loss {2:0.3f}'.format(epoch, np.mean(batch_train_acc), np.mean(batch_train_loss)))

In [ ]:
def gen_numbers_from_probability(nums_prob):

    ball_box = []

    for n in range(45):
        ball_count = int(nums_prob[n] * 100 + 1)
        ball = np.full((ball_count), n+1) #1부터 시작
        ball_box += list(ball)

    selected_balls = []

    while True:
        
        if len(selected_balls) == 6:
            break
        
        ball_index = np.random.randint(len(ball_box), size=1)[0]
        ball = ball_box[ball_index]

        if ball not in selected_balls:
            selected_balls.append(ball)

    return selected_balls

In [ ]:
# 마지막 회차까지 학습한 모델로 다음 회차 추론

print('receive numbers')

xs = x_samples[-1].reshape(1, 1, 45)

ys_pred = model.predict_on_batch(xs)

list_numbers = []

for n in range(10):
    numbers = gen_numbers_from_probability(ys_pred[0])
    print('{0} : {1}'.format(n, numbers))
    list_numbers.append(numbers)  